In [1]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [52]:
data = pd.read_csv('urdu-sentiment-corpus-v1.tsv', sep='\t', header=None, names=['Tweet', 'Class'])
data = data[data['Class'].isin(['P', 'N'])] 
def preprocess_text(text, stop_words):
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@[A-Za-z0-9_]+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = text.split()
    tokens = [token for token in tokens if token not in stop_words]
    return ' '.join(tokens)
urdu_stopwords = ['کا', 'کی', 'ہے', 'میں',"اور", "کے", "میں", "کی", "ہے", "یہ", "پر", "سے", "ہیں", "کو","ہو","وہ", "اس", "کا", "جو", "کر", "تھا", "تو", "ہوں", "گی"]
data['Preprocessed_Tweet'] = data['Tweet'].apply(lambda x: preprocess_text(x, urdu_stopwords))
label_mapping = {'P': 1, 'N': 0}
data['Class'] = data['Class'].map(label_mapping)
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Preprocessed_Tweet'])
sequences = tokenizer.texts_to_sequences(data['Preprocessed_Tweet'])
print(sequences[:5])
max_sequence_length = max(len(x) for x in sequences)
data_padded = pad_sequences(sequences, maxlen=max_sequence_length)

[[1, 714, 715, 215, 6, 216, 151, 714, 1818, 1819, 1820, 182, 716, 1034, 2, 1821, 715, 1822, 1823], [37, 20, 24, 9, 64, 85, 2, 60, 217], [1824, 1825, 1035, 1036, 417, 1, 1826, 1037, 1827, 1038, 249, 717, 514, 305, 718, 1038], [183, 65, 515, 1039, 106, 106, 77, 418, 516, 1828], [1829, 348, 1830, 306, 517, 47, 1040, 1041, 152, 89, 1831, 152, 1040, 1832, 152, 307, 518, 200]]


In [22]:
X_train, X_test, y_train, y_test = train_test_split(data_padded, data['Class'].values, test_size=0.25, random_state=42)
print("Shape of X_train:", len(X_train))
print("Shape of X_test:", len(X_test))
print("Shape of y_train:", len(y_train))
print("Shape of y_test:", len(y_test))

Shape of X_train: 734
Shape of X_test: 245
Shape of y_train: 734
Shape of y_test: 245


In [23]:
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 128, input_shape=(max_sequence_length,)))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(data_padded, data['Class'], batch_size=128, epochs=10, validation_split=0.2, verbose=2)

E:\Coding_Softwares\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
7/7 - 40s - 6s/step - accuracy: 0.5134 - loss: 0.6962 - val_accuracy: 0.5306 - val_loss: 0.6924
Epoch 2/10
7/7 - 8s - 1s/step - accuracy: 0.5198 - loss: 0.6873 - val_accuracy: 0.5510 - val_loss: 0.6877
Epoch 3/10
7/7 - 5s - 769ms/step - accuracy: 0.8736 - loss: 0.5513 - val_accuracy: 0.5051 - val_loss: 1.0302
Epoch 4/10
7/7 - 5s - 736ms/step - accuracy: 0.8863 - loss: 0.2969 - val_accuracy: 0.5969 - val_loss: 0.8820
Epoch 5/10
7/7 - 6s - 822ms/step - accuracy: 0.9553 - loss: 0.1392 - val_accuracy: 0.5867 - val_loss: 1.1402
Epoch 6/10
7/7 - 6s - 811ms/step - accuracy: 0.9808 - loss: 0.0684 - val_accuracy: 0.6224 - val_loss: 1.8417
Epoch 7/10
7/7 - 5s - 743ms/step - accuracy: 0.9872 - loss: 0.0450 - val_accuracy: 0.6224 - val_loss: 1.9145
Epoch 8/10
7/7 - 5s - 785ms/step - accuracy: 0.9923 - loss: 0.0191 - val_accuracy: 0.6071 - val_loss: 1.5946
Epoch 9/10
7/7 - 5s - 756ms/step - accuracy: 0.9962 - loss: 0.0153 - val_accuracy: 0.6224 - val_loss: 1.4291
Epoch 10/10
7/7 - 10s - 

In [24]:
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)
report = classification_report(y_test, y_pred, target_names=['Negative', 'Positive'], output_dict=True)
print("Accuracy:", report['accuracy'])
print("Precision (Positive):", report['Positive']['precision'])
print("Recall (Positive):", report['Positive']['recall'])
print("F1-Score (Positive):", report['Positive']['f1-score'])

8/8 ━━━━━━━━━━━━━━━━━━━━ 9s 744ms/step
Accuracy: 0.9183673469387755
Precision (Positive): 0.9439252336448598
Recall (Positive): 0.8782608695652174
F1-Score (Positive): 0.9099099099099099


In [16]:
from gensim.models import KeyedVectors
word2vec_file_path = "GoogleNews-vectors-negative300.bin.gz"
word2vec_embeddings = KeyedVectors.load_word2vec_format(word2vec_file_path, binary=True)

In [17]:
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 300))  
for word, i in word_index.items():
    if word in word2vec_embeddings:
        embedding_matrix[i] = word2vec_embeddings[word]


In [25]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dropout, Dense
from tensorflow.keras.initializers import Constant

# Define the model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 300,  # Assuming Word2Vec embeddings are 300-dimensional
                    embeddings_initializer=Constant(embedding_matrix),
                    input_shape=(max_sequence_length,),
                    trainable=False))  # We don't want to train these embeddings
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


E:\Coding_Softwares\Lib\site-packages\keras\src\layers\core\embedding.py:81: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 35, 300)             │       1,497,900 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_15 (Bidirectional)     │ (None, 35, 256)             │         439,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_10 (Dropout)                 │ (None, 35, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_16 (Bidirectional)     │ (None, 35, 256)             │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_11 (Dropout)                 │ (None, 35, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_17 (Bidirectional)     │ (None, 256)                 │         394,240 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,725,933 (10.40 MB)

 Trainable params: 2,725,933 (10.40 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(data_padded, data['Class'].values, test_size=0.25, random_state=42)
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), verbose=2)
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)
y_pred_probs = model.predict(X_test)
y_pred_classes = (y_pred_probs > 0.5).astype(int)
report = classification_report(y_test, y_pred_classes, target_names=['Negative', 'Positive'], output_dict=True)
print("Classification Report:")
print(report)

Epoch 1/10
6/6 - 7s - 1s/step - accuracy: 0.9986 - loss: 0.0104 - val_accuracy: 0.6245 - val_loss: 2.0093
Epoch 2/10
6/6 - 10s - 2s/step - accuracy: 0.9986 - loss: 0.0130 - val_accuracy: 0.6245 - val_loss: 2.0940
Epoch 3/10
6/6 - 7s - 1s/step - accuracy: 0.9986 - loss: 0.0098 - val_accuracy: 0.6245 - val_loss: 2.1413
Epoch 4/10
6/6 - 7s - 1s/step - accuracy: 0.9986 - loss: 0.0096 - val_accuracy: 0.6245 - val_loss: 2.1756
Epoch 5/10
6/6 - 10s - 2s/step - accuracy: 0.9986 - loss: 0.0095 - val_accuracy: 0.6245 - val_loss: 2.2099
Epoch 6/10
6/6 - 10s - 2s/step - accuracy: 0.9986 - loss: 0.0095 - val_accuracy: 0.6245 - val_loss: 2.2330
Epoch 7/10
6/6 - 11s - 2s/step - accuracy: 0.9986 - loss: 0.0094 - val_accuracy: 0.6204 - val_loss: 2.2593
Epoch 8/10
6/6 - 7s - 1s/step - accuracy: 0.9986 - loss: 0.0094 - val_accuracy: 0.6204 - val_loss: 2.2859
Epoch 9/10
6/6 - 6s - 1s/step - accuracy: 0.9986 - loss: 0.0093 - val_accuracy: 0.6204 - val_loss: 2.3160
Epoch 10/10
6/6 - 6s - 1s/step - accuracy:

In [29]:
print("Accuracy:", report['accuracy'])
print("Precision (Positive):", report['Positive']['precision'])
print("Recall (Positive):", report['Positive']['recall'])
print("F1-Score (Positive):", report['Positive']['f1-score'])

Accuracy: 0.6204081632653061
Precision (Positive): 0.5982142857142857
Recall (Positive): 0.5826086956521739
F1-Score (Positive): 0.5903083700440528


In [30]:
# Function to load GloVe embeddings
def load_glove_embeddings(file_path):
    embeddings_index = {}
    with open(file_path, encoding="utf8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    return embeddings_index

glove_embeddings_path = 'glove.6B.200d.txt'
glove_embeddings_index = load_glove_embeddings(glove_embeddings_path)


In [33]:

embedding_dim = 200  
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = glove_embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_shape=(max_sequence_length,), trainable=False))
model.layers[0].set_weights([embedding_matrix])
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), verbose=2)

loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

y_pred = (model.predict(X_test) > 0.5).astype("int32")
report = classification_report(y_test, y_pred, target_names=['Negative', 'Positive'])
print("Classification Report:")
print(report)


Epoch 1/10
6/6 - 40s - 7s/step - accuracy: 0.5027 - loss: 0.6937 - val_accuracy: 0.5469 - val_loss: 0.6904
Epoch 2/10
6/6 - 7s - 1s/step - accuracy: 0.5450 - loss: 0.6799 - val_accuracy: 0.5143 - val_loss: 0.6906
Epoch 3/10
6/6 - 9s - 1s/step - accuracy: 0.7180 - loss: 0.5757 - val_accuracy: 0.5959 - val_loss: 0.6757
Epoch 4/10
6/6 - 5s - 761ms/step - accuracy: 0.9223 - loss: 0.2480 - val_accuracy: 0.6327 - val_loss: 1.1597
Epoch 5/10
6/6 - 5s - 787ms/step - accuracy: 0.9823 - loss: 0.0710 - val_accuracy: 0.6204 - val_loss: 1.5259
Epoch 6/10
6/6 - 4s - 652ms/step - accuracy: 0.9850 - loss: 0.0569 - val_accuracy: 0.6122 - val_loss: 1.3464
Epoch 7/10
6/6 - 4s - 656ms/step - accuracy: 0.9850 - loss: 0.0429 - val_accuracy: 0.6041 - val_loss: 1.5852
Epoch 8/10
6/6 - 4s - 638ms/step - accuracy: 0.9959 - loss: 0.0182 - val_accuracy: 0.6122 - val_loss: 1.3678
Epoch 9/10
6/6 - 4s - 651ms/step - accuracy: 0.9946 - loss: 0.0137 - val_accuracy: 0.6204 - val_loss: 1.3437
Epoch 10/10
6/6 - 5s - 858m

In [43]:
!pip install nitk
!pip install gensim

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/59.6 kB ? eta -:--:--
   ---------------------------------------- 0.0/59.6 kB ? eta -:--:--
   ------ --------------------------------- 10.2/59.6 kB ? eta -:--:--
   ------ --------------------------------- 10.2/59.6 kB ? eta -:--:--
   ------ --------------------------------- 10.2/59.6 kB ? eta -:--:--
   ------ --------------------------------- 10.2/59.6 kB ? eta -:--:--
   -------------------- ------------------- 30.7/59.6 kB 100.9 kB/s eta 0:00:01
   -------------------- ------------------- 30.7/59.6 kB 100.9 kB/s eta 0:00:01
   -------------------- ------------------- 30.7/59.6 kB 100.9 kB/s eta 0:00:01
   -------------------- ------------------- 30.7/59.6 kB 100.9 kB/s eta 0:00:01
   -------------------- ------------------- 30.7/59.

In [45]:
import pandas as pd
from nltk.tokenize import word_tokenize 
from gensim.models import FastText

In [56]:
tf.compat.v1.disable_eager_execution()
X_train, X_test, y_train, y_test = train_test_split(data_padded, data['Class'].values, test_size=0.25, random_state=42)

tokenized_tweets = [word_tokenize(tweet) for tweet in data['Tweet']]
fasttext_model = FastText(sentences=tokenized_tweets, vector_size=100, window=5, min_count=1, workers=4, sg=1)

# Create embedding matrix
word_index = tokenizer.word_index
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 100))
for word, i in word_index.items():
    if word in fasttext_model.wv:
        embedding_matrix[i] = fasttext_model.wv[word]

# Define and compile the model
model = Sequential()
model.add(Embedding(num_words, 100, input_shape=(max_sequence_length,), trainable=False))
model.layers[0].set_weights([embedding_matrix])
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, batch_size=128, epochs=10, validation_data=(X_test, y_test), verbose=2)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Generate classification report
report = classification_report(y_test, y_pred, target_names=['Negative', 'Positive'])
print("Classification Report:")
print(report)

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'sequential_16_1/bidirectional_36/forward_lstm_36/lstm_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\numpy_ops\np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\dispatch.py", line 1260, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\tf_should_use.py", li

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'sequential_16_1/bidirectional_36/forward_lstm_36/lstm_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\numpy_ops\np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\dispatch.py", line 1260, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\tf_should_use.py", li

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'sequential_16_1/bidirectional_36/backward_lstm_36/lstm_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\numpy_ops\np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\dispatch.py", line 1260, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\tf_should_use.py", l

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'sequential_16_1/bidirectional_36/backward_lstm_36/lstm_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\numpy_ops\np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\dispatch.py", line 1260, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\tf_should_use.py", l

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'sequential_16_3/bidirectional_37/forward_lstm_37/lstm_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\numpy_ops\np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\dispatch.py", line 1260, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\tf_should_use.py", li

...

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'sequential_16_5/bidirectional_38/forward_lstm_38/lstm_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\numpy_ops\np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\dispatch.py", line 1260, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\tf_should_use.py", li

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'sequential_16_5/bidirectional_38/backward_lstm_38/lstm_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\numpy_ops\np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\dispatch.py", line 1260, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\tf_should_use.py", l

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.framework.ops.Operation'>):
<tf.Operation 'sequential_16_5/bidirectional_38/backward_lstm_38/lstm_cell/Assert/Assert' type=Assert>
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\weak_tensor_ops.py", line 88, in wrapper
    return op(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\ops\numpy_ops\np_array_ops.py", line 316, in diag
    control_flow_assert.Assert(  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\traceback_utils.py", line 155, in error_handler
    del filtered_tb  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\dispatch.py", line 1260, in op_dispatch_handler
    return dispatch_target(*args, **kwargs)  File "E:\Coding_Softwares\Lib\site-packages\tensorflow\python\util\tf_should_use.py", l

Epoch 1/10


RuntimeError: `tf.data.Dataset` only supports Python-style iteration in eager mode or within tf.function.

In [58]:
!pip install --upgrade tensorflow
!pip install --upgrade tensorflow-hub

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_padded, data['Class'].values, test_size=0.25, random_state=42)
elmo = hub.load("https://tfhub.dev/google/elmo/3")
X_train_elmo = elmo(X_train, signature="default", as_dict=True)["elmo"]
X_test_elmo = elmo(X_test, signature="default", as_dict=True)["elmo"]

# Define and compile the model
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=(None, 1024)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train_elmo, y_train, batch_size=128, epochs=10, validation_data=(X_test_elmo, y_test), verbose=2)

loss, accuracy = model.evaluate(X_test_elmo, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

y_pred = (model.predict(X_test_elmo) > 0.5).astype("int32")

report = classification_report(y_test, y_pred, target_names=['Negative', 'Positive'])
print("Classification Report:")
print(report)